In [9]:
import pandas as pd
import numpy as np

In [10]:
import os

In [14]:
print(os.listdir())

['test.csv', 'train.csv', 'train_test_csv.zip']


In [15]:
os.chdir("./assets")

FileNotFoundError: [WinError 2] The system cannot find the file specified: './assets'

In [16]:
test = pd.read_csv("test.csv")
print(test.head(1))

   id  battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
0   1           1043     1          1.8         1  14       0           5   

   m_dep  mobile_wt  ...  pc  px_height  px_width   ram  sc_h  sc_w  \
0    0.1        193  ...  16        226      1412  3476    12     7   

   talk_time  three_g  touch_screen  wifi  
0          2        0             1     0  

[1 rows x 21 columns]


In [17]:
train = pd.read_csv("train.csv")
print(train.head(1))

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   

   three_g  touch_screen  wifi  price_range  
0        0             0     1            1  

[1 rows x 21 columns]


In [18]:
print(test.columns)
print(train.columns)

Index(['id', 'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc',
       'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc',
       'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi'],
      dtype='object')
Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')


In [19]:
print(set(test.columns)-set(train.columns))
print(set(train.columns)-set(test.columns))

{'id'}
{'price_range'}


In [22]:
train_price_y = train["price_range"]
print(train_price_y[0:3])

0    1
1    2
2    2
Name: price_range, dtype: int64


In [25]:
test_x = test.drop(columns=["id"])
train_x = train.drop(columns=["price_range"])